In [3]:
# Comenzamos importando las librerías que usaremos
import pandas as pd
import requests
import re
import sqlite3
from bs4 import BeautifulSoup
import plotly.express as px


# Web scraping y creación de la base de datos

En este apartado obtenemos los poemas usando web scraping en la página web www.poesi.as. Creamos un base SQL para guardar la información obtenida en una tabla con la siguiente estructura: [autor - título - poema]

In [4]:
# Creamos un diccionario con el nombre de los autores y sus identificadores en la página web objetivo
dict_autores = {
    'Pedro_Salinas.htm': 'ps',
    'Jorge_Guillen.htm': 'jg',
    'Vicente_Aleixandre.htm': 'va',
    'Gerardo_Diego.htm': 'gd',
    'Federico_Garcia_Lorca.htm': 'fgl',
    'Emilio_Prados.htm': 'ep',
    'Damaso_Alonso.htm': 'da',
    'Rafael_Alberti.htm': 'ra',
    'Luis_Cernuda.htm': 'lc',
    'Manuel_Altolaguirre.htm': 'al'
}

# Especficamos la URL base
base_url = 'https://www.poesi.as/'

# Creamos una base de datos SQLite3
conn = sqlite3.connect('poemas.db')
cursor = conn.cursor()

# Creamos una tabla para almacenar el contenido de los poemas
cursor.execute('''CREATE TABLE IF NOT EXISTS poemas
                  (autor TEXT, titulo TEXT, poema TEXT)''')

# Recorremos el diccionario de autores y hacemos la solicitud HTTP
for autor, identificador in dict_autores.items():
    url_autor = base_url + autor
    response = requests.get(url_autor)
    soup = BeautifulSoup(response.content, 'html.parser')

    poem_counter = 0  # Contador para limitar a 100 poemas por autor

    for p_tag in soup.find_all('p', class_=['p1', 'p2', 'p3', 'p4']):
        if poem_counter >= 100:  # Si ya hemos extraído al menos 100 poemas, paramos el bucle
            break

        link_tag = p_tag.find('a')
        if link_tag and 'href' in link_tag.attrs:
            direccion_web = base_url + link_tag['href']

            # Hacemos una solicitud a la dirección web del poema
            response_poema = requests.get(direccion_web)
            soup_poema = BeautifulSoup(response_poema.content, 'html.parser')

            # Extraemos el título del poema
            titulo = link_tag.text.lower()

            # Extraemos el contenido del poema sin etiquetas <p> dentro de <div class="poema">
            contenido_poema = ''.join([p.text.strip() for p in soup_poema.select('div.poema p:not([class])')])

            # Realizamos las modificaciones en el texto del poema, para evitar errores de formato en los saltos de línea vacia en los poemas resultantes
            contenido_poema = contenido_poema.replace('\r\n', '\n')

            # Insertamos en la base de datos el título, el nombre del autor, el contenido del poema
            cursor.execute("INSERT INTO poemas (autor, titulo, poema) VALUES (?, ?, ?)",
                           (autor.lower().replace('_', ' ').replace('.htm', ''), titulo, contenido_poema.strip()))

            poem_counter += 1  # Incrementar el contador de poemas extraídos por autor

# Llevamos a cabo los cambios en la base de datos con Commit y cerramos la conexión
conn.commit()
conn.close()

print("Extracción y almacenamiento completados.")

Extracción y almacenamiento completados.


In [5]:
# Comprobamos que obtenemos 100 poemas para cada autor
# Comenzamos conectando a la base de datos poemas.db
conn_db1 = sqlite3.connect('poemas.db')
cursor_db1 = conn_db1.cursor()

# Calculamos el conteo de poemas para cada autor por medio de una función de agregación
cursor_db1.execute("SELECT autor, COUNT(*) FROM poemas GROUP BY autor")
conteo_poemas_db1 = cursor_db1.fetchall()

# Mostramos el conteo de poemas por autor
for autor, cantidad_poemas in conteo_poemas_db1:
    print(f"Autor: {autor}, Poemas: {cantidad_poemas}")

# Cerramos conexión a la primera base de datos
conn_db1.close()

Autor: damaso alonso, Poemas: 66
Autor: emilio prados, Poemas: 62
Autor: federico garcia lorca, Poemas: 100
Autor: gerardo diego, Poemas: 100
Autor: jorge guillen, Poemas: 100
Autor: luis cernuda, Poemas: 100
Autor: manuel altolaguirre, Poemas: 100
Autor: pedro salinas, Poemas: 100
Autor: rafael alberti, Poemas: 95
Autor: vicente aleixandre, Poemas: 100


# Exportar la base de datos a csv

In [9]:
# Creamos la conexión a la base de datos que hemos creado en el paso anterior
conn = sqlite3.connect('poemas.db')

# Consultamos todos los datos de la tabla por medio del comando SELECT *
query = 'SELECT * FROM poemas'

# Creamos el dataframe con la información de la consulta a la base de datos
df = pd.read_sql_query(query, conn)

# Cerramos la conexión a la base de datos
conn.close()

# Exportamos el DataFrame a un archivo CSV en Google Drive con el nombre db.csv
df.to_csv("/content/db.csv", index=False)
